In [1]:
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.utils import to_categorical
from keras import models, layers, optimizers
from os import environ

environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

Using Theano backend.


In [2]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [3]:
print('Train images shape:', train_images.shape)
print('Train labels shape:', train_labels.shape, '\n')
print('Test images shape:', test_images.shape)
print('Test labels shape:', test_labels.shape, '\n')

Train images shape: (60000, 28, 28)
Train labels shape: (60000,) 

Test images shape: (10000, 28, 28)
Test labels shape: (10000,) 



In [4]:

# for x in range(0, 1):
  #  plt.imshow(train_images[x], cmap='Greys')
   # plt.show()
   # print('Train image labels:', train_labels[x], '\n')


Training, Validation and Test dataset

In [5]:
valid_images = train_images[50000:60000]
valid_labels = train_labels[50000:60000]
train_images = train_images[0:50000]  # test images remain untouched
train_labels = train_labels[0:50000]

Creating Tensors

In [6]:
train_images = train_images.reshape((50000, 28, 28, 1))
train_images = train_images.astype('float32') / 255  # converting to a [0, 1] scale
valid_images = valid_images.reshape((10000, 28, 28, 1))
valid_images = valid_images.astype('float32') / 255  # converting to a [0, 1] scale
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255  # converting to a [0, 1] scale

In [7]:
print('A label:', train_labels[19])
train_labels = to_categorical(train_labels)
valid_labels = to_categorical(valid_labels)
test_labels = to_categorical(test_labels)
print('A one-hot encode label:', train_labels[19])

A label: 9
A one-hot encode label: [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]


CNN Architeture

In [8]:
my_model = models.Sequential()
my_model.add(layers.Conv2D(filters=16, kernel_size=(3, 3), strides=(1, 1),
                           use_bias=True, input_shape=(28, 28, 1)))
my_model.add(layers.Activation('relu'))
my_model.add(layers.Conv2D(filters=16, kernel_size=(3, 3), strides=(1, 1), use_bias=True))
my_model.add(layers.Activation('relu'))

my_model.add(layers.MaxPooling2D(pool_size=(2, 2)))

my_model.add(layers.Dropout(rate=0.4))

my_model.add(layers.Conv2D(filters=8, kernel_size=(3, 3), strides=(1, 1), use_bias=True))
my_model.add(layers.Activation('relu'))

my_model.add(layers.MaxPooling2D(pool_size=(2, 2)))

my_model.add(layers.Flatten())

my_model.add(layers.Dropout(rate=0.4))

my_model.add(layers.Dense(units=10, use_bias=True))
my_model.add(layers.Activation('relu'))

my_model.add(layers.Dense(units=10, use_bias=True))
my_model.add(layers.Activation('softmax'))

CNN Summary (Mostra os dados da rede neural configurada)

In [9]:
my_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 16)        160       
_________________________________________________________________
activation_1 (Activation)    (None, 26, 26, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 16)        2320      
_________________________________________________________________
activation_2 (Activation)    (None, 24, 24, 16)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 8)         1160      
__________

CNN Loss and Optimizer

In [10]:
compile = my_model.compile(optimizers.sgd(lr=0.1, decay=0.01), loss='categorical_crossentropy',
                           metrics=['accuracy'])

CNN Training

In [11]:
fit = my_model.fit(x=train_images, y=train_labels, batch_size=2500, epochs=20,
                   validation_data=(valid_images, valid_labels))

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 67s 1ms/step - loss: 2.2941 - acc: 0.1273 - val_loss: 2.2785 - val_acc: 0.2051
Epoch 2/20
50000/50000 [==============================] - 69s 1ms/step - loss: 2.2685 - acc: 0.1631 - val_loss: 2.2373 - val_acc: 0.2370
Epoch 3/20
50000/50000 [==============================] - 74s 1ms/step - loss: 2.2204 - acc: 0.1879 - val_loss: 2.1397 - val_acc: 0.2926
Epoch 4/20
50000/50000 [==============================] - 74s 1ms/step - loss: 2.1233 - acc: 0.2348 - val_loss: 1.9637 - val_acc: 0.4250
Epoch 5/20
50000/50000 [==============================] - 74s 1ms/step - loss: 1.9799 - acc: 0.3146 - val_loss: 1.7436 - val_acc: 0.5418
Epoch 6/20
50000/50000 [==============================] - 70s 1ms/step - loss: 1.8059 - acc: 0.3932 - val_loss: 1.5173 - val_acc: 0.5787
Epoch 7/20
50000/50000 [==============================] - 66s 1ms/step - loss: 1.6464 - acc: 0.4449 - val_loss: 1.3302 - val_acc

Save Model

In [13]:
my_model.save(filepath=r'/home/daniel/Documents/treino', overwrite=True)